In [ ]:
import pandas as pd
import os
import shutil
import numpy as np
import glob

In [ ]:
idat_path = '/data/CARD/PD/GP2/raw_genotypes/GP2_idats'
swarm_scripts_dir = f'/data/CARD/PD/GP2/swarm_scripts'

ilmn_files_path = '/data/CARD/PD/GP2/ilmn_files'
# A1 is hg19, A2 is hg38. csv needed for indel calls
bpm_csv = f'{ilmn_files_path}/NeuroBooster_20042459_A2.csv' 
bpm = f'{ilmn_files_path}/NeuroBooster_20042459_A2.bpm'
egt = f'{ilmn_files_path}/recluster_09272022.egt'
iaap = f'{ilmn_files_path}/iaap-cli/iaap-cli'

ref_fasta = '/data/vitaled2/ref/GRCh38/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna'

chroms = [str(i) for i in range(1,23)] + ['X','Y','M']


key_file = '/data/CARD/PD/GP2/clinical/key_merge/GP2_merge/GP2_master_key_full.txt'
key = pd.read_csv(f'{key_file}', sep='\t')
key.loc[:,'filename'] = key.loc[:,'SentrixBarcode_A'].astype(str) + '_' + key.loc[:,'SentrixPosition_A'].astype(str)
cohorts = [
    'SYNAPS-KZ', 
    'CORIELL', 
    'IPDGCEA-JU', 
    'KOC', 
    'IPDGCAF-NG', 
    'IPDGCEA-MU', 
    'PAGE', 
    'NZP3', 
    'S4', 
    'BCM',
    'BLAACPD-RUSH', 
    'BIOFIND', 
    'UMD', 
    'BLAACPD-UAB', 
    'LCC', 
    'SYNAPS-GR', 
    'MDGAP-QSBB', 
    'APGS', 
    'MDGAP-KINGS', 
    'MDGAP-IBB', 
    'BLAACPD-UC', 
    'BLAACPD-KPM', 
    'PPMI', 
    'IPDGCEA-UKM'
]

release_key = key.loc[key['study'].isin(cohorts)]
release_key.to_csv('/data/CARD/PD/GP2/clinical/key_merge/GP2_merge/release3_key.csv', index=False, header=True)

samples_list = release_key.loc[:,'filename'].unique()
barcodes_list = list(set([x.split('_')[0] for x in samples_list]))

genes = pd.read_csv('/data/CARD/PD/GP2/ref_panel/glist-hg38', sep='\s+', header=None, names=['chr','start','end','symbol'], dtype={'chr':str,'start':int,'end':int})
genes.columns = ['CHR','START','STOP','NAME']
genes_out = genes.loc[genes.CHR.isin(chroms)]
genes_out[['NAME','CHR','START','STOP']].to_csv('/data/CARD/PD/GP2/ref_panel/glist_hg38_intervals.csv', index=False)
# gene_list originally from plink glist
gene_list = '/data/CARD/PD/GP2/ref_panel/glist_hg38_intervals.csv'

raw_geno_path = '/data/CARD/PD/GP2/raw_genotypes'
snp_metrics_path = f'{raw_geno_path}/GP2_snp_metrics'
idat_path = f'{raw_geno_path}/GP2_idats'
plink_file_path = f'{raw_geno_path}/GP2_plink'

In [ ]:
len(samples_list)

In [ ]:
# get snp metrics using new cluster file
bcftools_plugins_path = '/data/vitaled2/bin'
with open(f'{swarm_scripts_dir}/snp_metrics.swarm', 'w') as f:
    for code in barcodes_list:
        idat_path_ = f'{idat_path}/{code}'
        metrics_out = f'{snp_metrics_path}'

        cmd = f'\
python /data/vitaled2/GenoTools/run_snp_metrics_pipeline.py \
--idat_path {idat_path_} \
--bpm {bpm} \
--bpm_csv {bpm_csv} \
--egt {egt} \
--ref_fasta {ref_fasta} \
--out_path {metrics_out} \
--iaap {iaap} \
--bcftools_plugins_path {bcftools_plugins_path}'
        f.write(f'{cmd}\n')
f.close()

!swarm -f {swarm_scripts_dir}/snp_metrics.swarm -g 16 -t 16 --time=04:00:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

In [ ]:
# check if snp metrics produced across all samples
completed_list = []
missing_list = []
for sample in samples_list:
    code = sample.split('_')[0]
    metrics_path = f'{snp_metrics_path}/{code}'
    for chrom in chroms:
        mfile = f'{metrics_path}/snp_metrics_{sample}_chr{chrom}.csv'
        if os.path.isfile(mfile):
            completed_list.append(sample)
        else:
            missing_list.append(sample)
completed_final = list(set(completed_list))
missing_final =list(set(missing_list))
        
    

In [ ]:
print(len(completed_final))
print(len(missing_final))

In [ ]:
# now check if we have idats for missing snp metrics
missing_idats = []
exists_idats = []
for sample in missing_final:
    code = sample.split('_')[0]
    grn = f'{idat_path}/{code}/{sample}_Grn.idat'
    red = f'{idat_path}/{code}/{sample}_Red.idat'
    if os.path.isfile(grn) & os.path.isfile(red):
        exists_idats.append(sample)
    else:
        missing_idats.append(sample)

In [ ]:
print(len(missing_idats))
print(len(exists_idats))

In [ ]:
cnv_path = f'/data/CARD/PD/GP2/raw_genotypes/GP2_cnvs/release3'
# using just autosomes for CNV analysis due to call quality
chroms = [str(i) for i in range(1,23)]

ancestry_labels = [x.split('/')[-1].replace('.bed','').split('_')[-1] for x in glob.glob('/data/CARD/PD/GP2/genotypes/GP2/round3/clean/GP2_round3_NEW_MERGE*.bed') if '_maf_hwe' not in x]

release_key[['FID','GP2sampleID']].to_csv(f'{cnv_path}/release3.samples', sep='\t', header=False, index=False)
release_key[['GP2sampleID','IID']].to_csv(f'{cnv_path}/release3_sample_id_key.csv')
release_covars = release_key.loc[:,['FID', 'GP2sampleID','sex_for_qc', 'age', 'age_of_onset']]

In [ ]:
# snp QC for cnvs
for label in ancestry_labels:
    out_name = f'GP2_round3_NEW_MERGE_{label}'
    out_path = f'{cnv_path}/{out_name}'
    geno_path = f'/data/CARD/PD/GP2/genotypes/GP2/round3/clean/{out_name}'

    cmd1 = f'\
module load plink/1.9; plink \
--bfile {geno_path} \
--keep {cnv_path}/release3.samples \
--make-bed \
--out {out_path}'

    cmd2 = f'\
module load plink/1.9; plink \
--bfile {out_path} \
--maf 0.01 \
--geno 0.02 \
--hwe 5e-6 \
--autosome \
--make-bed \
--out {out_path}_clean'

    cmd3 = f'\
module load plink/1.9; plink \
--bfile {out_path}_clean \
--pca \
--out {out_path}_clean_pcs'

    cmds = [cmd1, cmd2, cmd3]

    for cmd in cmds:
        !{cmd}

    pcs = pd.read_csv(f'{out_path}_clean_pcs.eigenvec', sep='\s+')
    pc_num = pcs.iloc[:, 2:].shape[1]
    pc_names = ['FID','GP2sampleID'] + [f'PC{i}' for i in range(1, pc_num+1)]
    pcs.columns = pc_names

    cov = pcs.merge(release_covars, on=['FID','GP2sampleID'], how='left')
    cov.age.fillna(cov.age.mean(), inplace=True)
    cov.age_of_onset.fillna(cov.age_of_onset.mean(), inplace=True)
    cov.sex_for_qc.fillna(cov.sex_for_qc.median(), inplace=True)
    cov.rename(columns={'GP2sampleID':'sampleid','sex_for_qc':'sex'}, inplace=True)
    cov.to_csv(f'{out_path}_clean.cov', sep='\t', header=True, index=False)

    samples = cov.merge(release_key[['GP2sampleID','IID']], left_on='sampleid', right_on='GP2sampleID', how='left')
    samples['IID'].to_csv(f'{out_path}_clean_barcode.samples', header=False, index=False)

    bim = pd.read_csv(f'{out_path}_clean.bim', sep='\s+', header=None, names=['chr','id','pos','bp','a1','a2'], usecols=['id'])
    bim.to_csv(f'{out_path}_clean.snps', sep='\t', header=False, index=False)
        

        
# skip ld pruning until we have better quality rare calls
# --exclude exclusion_regions_hg38.txt \


#         cmd3 = f'\
# plink \
# --bfile {geno}_release2_maf_geno_hwe \
# --indep-pairwise 1000 10 0.02 \
# --autosome \
# --out {geno}_release2_prune'
    
#         cmd4 = f'\
# plink \
# --bfile {geno}_release2_maf_geno_hwe \
# --extract {geno}_release2_prune.prune.in \
# --make-bed \
# --out {geno}_release2_clean'

In [ ]:
with open(f'{swarm_scripts_dir}/cnvs.swarm', 'w') as f:
    for label in ancestry_labels:
        out_name = f'GP2_round3_NEW_MERGED_{label}'
        out_path = f'{cnv_path}/{out_name}'
#         geno_path = f'/data/CARD/PD/GP2/genotypes/GP2/round3/clean/{out_name}'
        samples = pd.read_csv(f'{out_path}_clean_barcode.samples', header=None, names=['IID'])
        
        for sample in samples.IID.unique():
            code = sample.split('_')[0]
            for chrom in chroms:
            
                mfile = f'{snp_metrics_path}/{code}/snp_metrics_{sample}_chr{chrom}.csv'

                out_file = f'{cnv_path}/CNV_{label}_{sample}_chr{chrom}.csv'
                intervals = f'/data/CARD/PD/GP2/ref_panel/glist_hg38_chr{chrom}.csv'

                cmd = f'\
python /data/vitaled2/GenoTools/run_cnv_pipeline.py \
--metrics {mfile} \
--out_path {out_file} \
--intervals {intervals} \
--min_variants 10 \
--kb_window 250'
            
                f.write(f'{cmd}\n')
f.close()

In [ ]:
!swarm -f {swarm_scripts_dir}/cnvs.swarm -g 4 -t 8 --time=00:30:00 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

In [ ]:
# write dosages per chromosome per ancestry per cnv-type
cnv_types = ['PERCENT_BAF_INSERTION','PERCENT_L2R_DELETION','PERCENT_L2R_DUPLICATION']

with open(f'{swarm_scripts_dir}/cnv_dosages.swarm', 'w') as f:
    for label in ancestry_labels:
        for chrom in chroms:

            cnv_files_list = glob.glob(f'{cnv_path}/CNV_{label}_*_chr{chrom}.csv')
            cnv_files_df = pd.DataFrame({'filename':cnv_files_list})
            cnv_files_df.to_csv(f'{cnv_path}/CNV_{label}_chr{chrom}_files.csv', header=False, index=False)

            for cnv_type in cnv_types:
                
                cmd = f'\
python /data/vitaled2/GenoTools/run_cnv_dosage_pipeline.py \
--files {cnv_path}/CNV_{label}_chr{chrom}_files.csv \
--label {label} \
--chrom {chrom} \
--cnv_type {cnv_type} \
--out_path {cnv_path}/CNV_{label}_chr{chrom}_{cnv_type}.csv'
                f.write(f'{cmd}\n')
f.close()
            


In [ ]:
!swarm -f {swarm_scripts_dir}/cnv_dosages.swarm -g 32 -t 16 --logdir {swarm_scripts_dir}/logs --gres=lscratch:20 --partition=norm

In [ ]:
with open(f'{swarm_scripts_dir}/update_cnv_ids.swarm','w') as f:
    for label in ancestry_labels:
        for dose in cnv_types:
            for chrom in chroms:

                dosagefile = f'{cnv_path}/CNV_{label}_chr{chrom}_{cnv_type}.csv'
                dosagefile_out = f'{cnv_path}/CNV_{label}_chr{chrom}_{cnv_type}_gp2ids.csv'
                pheno_out = f'{cnv_path}/GP2_round2_{label}_chr{chrom}_{cnv_type}.pheno'
                cmd = f'\
python /data/vitaled2/GenoTools/update_cnv_ids.py \
--dosagefile {dosagefile} \
--key /data/CARD/PD/GP2/clinical/key_merge/GP2_merge/release3_key.csv \
--pheno_out {pheno_out} \
--out_path {dosagefile_out}'
                f.write(f'{cmd}\n')
f.close()